# Goal

I wish to cleanup the datetime columns of this data, turning it into proper timeseries format. In doing this I also wanted to add new columns, labelling the day of the week that the incidents occurred on and whether or not the incidents occurred on holidays. The reason for this is that I wanted to test the hypotheses that the distribution of laser related incidents will not be equal across the days of the week, and that the frequency of incidents will differ on holidays. Before conducting these statistical analyses I needed to get the data into a more usable format.

## set up workspace
import modules, note the kaggle kernels lack the holidays module
pip install holidays

In [1]:
from datetime import date, datetime
import calendar
import pandas as pd
from pandas import Series, DataFrame
import holidays

I merged all the .csv files into one, removing the 'Total' row for the bottom of each. This was done using unix commands: 

echo 'DATE,TIME (UTC),ACID,No. A/C,TYPE A/C,ALT,MAJOR CITY,COLOR,Injury Reported,CITY,STATE' > all_laser_dat.csv

grep -v 'Total' *.csv >> all_laser_dat.csv


In [5]:
laser_dat = pd.read_csv('all_laser_dat.csv')
laser_dat.head()

,DATE,TIME (UTC),ACID,No. A/C,TYPE A/C,ALT,MAJOR CITY,COLOR,Injury Reported,CITY,STATE
0,01-Jan-10,102,AIR1,1,HELO,2000,LEX,Green,NO,Lexington,Kentucky
1,01-Jan-10,403,ASA513,1,B737,10000,LAX,Green,NO,Los Angeles,California
2,01-Jan-10,246,EGF3002,1,E135,7000,LAX,Green,NO,Los Angeles,California
3,01-Jan-10,157,EGF3086,1,E135,7000,LAX,Green,NO,Los Angeles,California
4,01-Jan-10,258,JBU300,1,A320,12500,SLI,Green,NO,Los Alamitos,California


with the dataframe built I removed the 4 rows that do not have a time associated with them

In [6]:
laser_dat = laser_dat[laser_dat['TIME (UTC)'] != 'UNKN']

## Turn the dates to python datetime format
using the time and the data columns, and a few intermediate steps

In [7]:
laser_dat['TIME (UTC)']

laser_dat['hour'] = laser_dat.apply(lambda x: x['TIME (UTC)'][:-2], axis=1)
laser_dat['min'] = laser_dat.apply(lambda x: x['TIME (UTC)'][-2:], axis=1)

laser_dat['min'].fillna(0, inplace=True)
laser_dat['hour'].fillna(0, inplace=True)

#account for lack of zeros
min_changed = []
for i in laser_dat['min']:
	if len(i) == 0:
		min_changed.append('00')
	elif len(i) == 1:
		min_changed.append('0'+i)
	else:
		min_changed.append(i)

hr_changed = []
for i in laser_dat['hour']:
	if len(i) == 0:
		hr_changed.append('00')
	elif len(i) == 1:
		hr_changed.append('0'+i)
	else:
		hr_changed.append(i)


laser_dat['min_adj'] = min_changed
laser_dat['hr_adj'] = hr_changed

laser_dat['time'] = laser_dat.apply(lambda x: '%s:%s:%s' % (x['DATE'] , x['hr_adj'],  x['min_adj'] ), axis=1)

laser_dat['date_time'] = laser_dat.apply(lambda x: datetime.strptime(x['time'], '%d-%b-%y:%H:%M'), axis=1)

#drop the making of datetime columns, except for the 'hour' column
laser_dat = laser_dat.drop(['time','hour','min', 'min_adj','TIME (UTC)','DATE'], axis=1)


## Add a column with the day of the week
This uses the calendar library to get the day of the week for each date in the df

In [8]:
laser_dat['day_of_week'] = laser_dat.apply(lambda x:  calendar.day_name[x['date_time'].weekday()] , axis = 1)

## Add a column with holiday/no holidays
This uses the holidays library

In [9]:
#get us holidays
us_holidays = holidays.UnitedStates()
#build the column
holiday_tf = []
for date in laser_dat['date_time']:
	if date in us_holidays:
		holiday_tf.append(True)
	elif date not in us_holidays:
		holiday_tf.append(False)


laser_dat['holidays'] = holiday_tf

In [10]:
laser_dat

,ACID,No. A/C,TYPE A/C,ALT,MAJOR CITY,COLOR,Injury Reported,CITY,STATE,hr_adj,date_time,day_of_week,holidays
0,AIR1,1,HELO,2000,LEX,Green,NO,Lexington,Kentucky,01,2010-01-01 01:02:00,Friday,True
1,ASA513,1,B737,10000,LAX,Green,NO,Los Angeles,California,04,2010-01-01 04:03:00,Friday,True
2,EGF3002,1,E135,7000,LAX,Green,NO,Los Angeles,California,02,2010-01-01 02:46:00,Friday,True
3,EGF3086,1,E135,7000,LAX,Green,NO,Los Angeles,California,01,2010-01-01 01:57:00,Friday,True
4,JBU300,1,A320,12500,SLI,Green,NO,Los Alamitos,California,02,2010-01-01 02:58:00,Friday,True
5,JBU303,1,A320,2000,LGB,Green,NO,Los Angeles,California,02,2010-01-01 02:55:00,Friday,True
6,PD2,1,E120,700,FAT,Green,NO,Fresno,California,08,2010-01-01 08:57:00,Friday,True
7,SKW6083,1,CRJ2,6800,LNK,Green,NO,Lincoln,Nebraska,00,2010-01-01 00:44:00,Friday,True
8,SKW6318,1,E120,1000,LAX,Green,NO,Los Angeles,California,01,2010-01-01 01:55:00,Friday,True
9,SKW6433,1,CRJ2,1000,FAT,Green,NO,Fresno,California,07,2010-01-01 07:30:00,Friday,True


Now the data has a column with the data in python datetime format, as well as two additional columns 'day_of_week' and 'holidays' which I will use for visualization and statistics.

In [ ]:
laser_dat.to_csv('adjusted_laser_data.csv')